[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mmcmanus1/rlhf-canary/blob/main/notebooks/02_profiler_deep_dive.ipynb)

# Profiler Deep Dive: Diagnosing Training Slowdowns

Use PyTorch's built-in profiler to pinpoint exactly where your training time is going. This notebook shows you how to identify GPU bottlenecks, CPU overhead, and memory transfer issues.

**What you'll learn:**
1. How to enable profiling in your canary configs
2. Running a profiled training job
3. Interpreting profiler output (CUDA vs CPU time)
4. Identifying top operations consuming time
5. Exporting traces for TensorBoard visualization
6. Practical workflow: "My run got slower - here's how to find why"

**Requirements:** GPU runtime (Runtime > Change runtime type > T4 GPU)

**Runtime:** ~10-15 minutes

## 1. Setup

In [ ]:
import os
import re
import sys

print("Starting Environment Setup...")

# --- 1. Clone or update the repo ---
if not os.path.exists("/content/rlhf-canary"):
    !git clone https://github.com/mmcmanus1/rlhf-canary.git /content/rlhf-canary
else:
    !cd /content/rlhf-canary && git pull --ff-only

%cd /content/rlhf-canary

# --- 2. Force-Install the "Safe Harbor" Stack ---
!pip install "trl==0.11.4" "transformers==4.44.2" "peft==0.12.0" "accelerate==0.34.2" "tokenizers==0.19.1" --force-reinstall --no-deps --quiet
!pip install -q datasets pydantic click PyYAML bitsandbytes
print("Libraries installed (TRL 0.11.4 / Transformers 4.44.2)")

# --- 3. Patch pyproject.toml (Prevent future drift) ---
project_file = "/content/rlhf-canary/pyproject.toml"
if os.path.exists(project_file):
    with open(project_file, "r") as f:
        content = f.read()
    
    if "trl==0.11.4" not in content:
        content = re.sub(r'trl[<>=!~]+[\d\.]+', 'trl==0.11.4', content)
        with open(project_file, "w") as f:
            f.write(content)
        print("Config file patched to lock TRL 0.11.4")

# --- 4. Patch Source Code (Compatibility Fix) ---
runner_file = "/content/rlhf-canary/canary/runner/local.py"
if os.path.exists(runner_file):
    with open(runner_file, "r") as f:
        code = f.read()
    
    if "processing_class=" in code:
        code = code.replace("processing_class=", "tokenizer=")
        with open(runner_file, "w") as f:
            f.write(code)
        print("Code patched: Reverted 'processing_class' to 'tokenizer'")
    else:
        print("Code is already compatible.")

# --- 5. Install the package ---
!pip install -e . --quiet

print("Environment Ready!")

In [ ]:
# Verify GPU and installation
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

import canary
print(f"Canary module loaded from: {canary.__file__}")

## 2. Understanding the Profiler

The RLHF Canary profiler captures detailed timing information during training:

- **CUDA operations**: GPU kernel execution times (matrix multiplications, attention, etc.)
- **CPU operations**: Python/C++ operations (data loading, tokenization, etc.)
- **Memory profiling**: Memory allocation patterns

### Profiler Configuration

```yaml
profiler:
  enabled: true            # Turn on profiling
  start_step: 50           # Start capturing at step 50 (skip warmup)
  num_steps: 20            # Capture 20 steps
  output_dir: ./profiler   # Where to save traces
  record_shapes: true      # Record tensor shapes
  profile_memory: true     # Track memory allocations
  with_stack: false        # Stack traces (slow, disable for speed)
```

**Key insight**: We skip the first ~50 steps because they include one-time costs (JIT compilation, memory allocation) that aren't representative of steady-state performance.

## 3. Create a Profiling Configuration

Let's create a config that enables the profiler. We'll use a shorter run (80 steps) and profile steps 50-70.

In [ ]:
profiler_config = """
name: dpo_profiled
description: DPO training with profiler enabled for performance analysis

# Model configuration
model_name: EleutherAI/pythia-70m
use_peft: true
lora_r: 16
lora_alpha: 32
lora_dropout: 0.05

# Training configuration
training_type: dpo
max_steps: 80
batch_size: 2
gradient_accumulation_steps: 4
learning_rate: 5.0e-5
max_length: 256
warmup_steps: 10

# DPO-specific
beta: 0.1
max_prompt_length: 64

# Dataset configuration
dataset_name: Anthropic/hh-rlhf
dataset_split: train
dataset_size: 512
seed: 42

# Output configuration
output_dir: ./profiler_output
metrics_warmup_steps: 10

# PROFILER ENABLED!
profiler:
  enabled: true
  start_step: 50      # Start after warmup
  num_steps: 20       # Profile 20 steps
  output_dir: ./profiler_traces
  record_shapes: true
  profile_memory: true
  with_stack: false   # Keep false for speed
"""

with open('configs/dpo_profiled.yaml', 'w') as f:
    f.write(profiler_config)

print("Created configs/dpo_profiled.yaml with profiler enabled")
print("\nProfiler will capture steps 50-70 (20 steps total)")

## 4. Run Training with Profiling

This will take ~8-10 minutes. The profiler adds minimal overhead (~5%).

In [ ]:
!python -m canary.cli run configs/dpo_profiled.yaml -o ./profiler_output/run1

## 5. Analyze Profiler Output

The profiler produces:
1. **ProfilerSummary** in metrics.json - High-level stats
2. **trace.json** - Detailed trace for TensorBoard

In [ ]:
import json
from pathlib import Path

# Find the metrics file
metrics_paths = list(Path('./profiler_output/run1').rglob('metrics.json'))
if not metrics_paths:
    raise FileNotFoundError("No metrics.json found. Did the training run complete?")

metrics_path = metrics_paths[0]
print(f"Metrics file: {metrics_path}")

with open(metrics_path) as f:
    metrics = json.load(f)

# Check if profiler summary exists
if 'profiler' in metrics and metrics['profiler']:
    profiler_summary = metrics['profiler']
    print("\n" + "="*60)
    print("PROFILER SUMMARY")
    print("="*60)
    print(f"\nTrace file: {profiler_summary.get('trace_path', 'N/A')}")
    print(f"\nTotal CUDA time: {profiler_summary.get('cuda_time_total_ms', 0):.2f} ms")
    print(f"Total CPU time:  {profiler_summary.get('cpu_time_total_ms', 0):.2f} ms")
    print(f"Self CUDA time:  {profiler_summary.get('self_cuda_time_total_ms', 0):.2f} ms")
else:
    print("No profiler summary found in metrics")

In [ ]:
# Show top CUDA operations (GPU bottlenecks)
if 'profiler' in metrics and metrics['profiler']:
    profiler_summary = metrics['profiler']
    
    print("\n" + "="*60)
    print("TOP 10 CUDA OPERATIONS (GPU time)")
    print("="*60)
    print(f"{'Operation':<40} {'Time (ms)':>10} {'Count':>8}")
    print("-"*60)
    
    for op in profiler_summary.get('top_cuda_ops', [])[:10]:
        name = op['name'][:38] if len(op['name']) > 38 else op['name']
        print(f"{name:<40} {op['self_cuda_time_ms']:>10.2f} {op['count']:>8}")
    
    print("\n" + "="*60)
    print("TOP 10 CPU OPERATIONS (CPU time)")
    print("="*60)
    print(f"{'Operation':<40} {'Time (ms)':>10} {'Count':>8}")
    print("-"*60)
    
    for op in profiler_summary.get('top_cpu_ops', [])[:10]:
        name = op['name'][:38] if len(op['name']) > 38 else op['name']
        print(f"{name:<40} {op['self_cpu_time_ms']:>10.2f} {op['count']:>8}")

## 6. Interpreting the Results

### What to look for:

**CUDA operations (GPU)**:
- `aten::mm`, `aten::bmm` - Matrix multiplications (expected to be high)
- `aten::_scaled_dot_product_flash_attention` - Attention (expected to be high)
- `aten::copy_` - Memory transfers (should NOT dominate - indicates CPU/GPU sync issues)

**CPU operations**:
- `aten::to`, `aten::_to_copy` - Tensor type conversions (high = dtype issues)
- `Optimizer.step` - Should be reasonable, not dominant
- High CPU time with low GPU util = dataloader/tokenization bottleneck

### Warning signs:
1. **Memory copies dominating**: CPU<->GPU transfers are slow
2. **`to` operations high**: Dtype mismatches causing conversions
3. **CPU >> CUDA time**: GPU is idle waiting for data
4. **Single operation dominates**: Optimization opportunity

In [ ]:
# Quick diagnosis based on profiler results
if 'profiler' in metrics and metrics['profiler']:
    profiler_summary = metrics['profiler']
    
    cuda_time = profiler_summary.get('cuda_time_total_ms', 0)
    cpu_time = profiler_summary.get('cpu_time_total_ms', 0)
    
    print("\n" + "="*60)
    print("QUICK DIAGNOSIS")
    print("="*60)
    
    # Check CPU vs GPU balance
    if cpu_time > 0 and cuda_time > 0:
        ratio = cpu_time / cuda_time
        if ratio > 2.0:
            print(f"\nWARNING: CPU time ({cpu_time:.0f}ms) >> CUDA time ({cuda_time:.0f}ms)")
            print("This suggests a CPU bottleneck (dataloader, tokenization, or Python overhead)")
        elif ratio < 0.5:
            print(f"\nGOOD: CUDA time ({cuda_time:.0f}ms) >> CPU time ({cpu_time:.0f}ms)")
            print("GPU is well-utilized, training is compute-bound (optimal)")
        else:
            print(f"\nBALANCED: CPU time ({cpu_time:.0f}ms) ~ CUDA time ({cuda_time:.0f}ms)")
            print("Mixed workload - both CPU and GPU contribute significantly")
    elif cuda_time == 0 and cpu_time > 0:
        print(f"\nINFO: No CUDA kernel times recorded (CPU time: {cpu_time:.0f}ms)")
        print("This can happen when CUPTI profiling is unavailable (common on some cloud GPUs)")
        print("CPU-side CUDA calls were still captured - see CPU operations above")
    
    # Check for memory copy dominance
    top_cuda = profiler_summary.get('top_cuda_ops', [])
    if top_cuda:
        copy_ops = [op for op in top_cuda if 'copy' in op['name'].lower()]
        total_copy_time = sum(op['self_cuda_time_ms'] for op in copy_ops)
        total_cuda_time = sum(op['self_cuda_time_ms'] for op in top_cuda)
        
        if total_cuda_time == 0:
            print("\nNo CUDA kernel times recorded")
            print("This can happen when CUPTI profiling is unavailable (common on some cloud GPUs)")
            print("CPU-side CUDA calls were still captured - see CPU operations above")
        elif total_copy_time / total_cuda_time > 0.3:
            print(f"\nWARNING: Memory copies are {total_copy_time/total_cuda_time:.0%} of CUDA time")
            print("Consider: reducing CPU<->GPU transfers, using pinned memory")
        else:
            print(f"\nMemory copy overhead: {total_copy_time/total_cuda_time:.0%} of CUDA time (acceptable)")

## 7. Export Trace for TensorBoard

For deeper analysis, you can visualize the trace in TensorBoard.

In [ ]:
# Find the trace file
trace_files = list(Path('./profiler_traces').rglob('trace.json'))

if trace_files:
    trace_path = trace_files[0]
    print(f"Trace file found: {trace_path}")
    print(f"File size: {trace_path.stat().st_size / 1024:.1f} KB")
    
    print("\n" + "="*60)
    print("TO VIEW IN TENSORBOARD:")
    print("="*60)
    print("\n1. Download the trace file to your local machine")
    print("2. Run: tensorboard --logdir=./profiler_traces")
    print("3. Go to the 'PyTorch Profiler' tab")
    print("\nOr view in Chrome:")
    print("1. Open chrome://tracing")
    print("2. Load the trace.json file")
else:
    print("No trace file found")

## 8. Practical Workflow: Diagnosing a Slowdown

Let's simulate a real debugging scenario. First, let's get baseline metrics.

In [ ]:
# Save the current run as baseline for comparison
baseline_metrics = metrics.copy()

print("Baseline metrics:")
print(f"  Step time (mean): {baseline_metrics['perf']['step_time']['mean']:.4f}s")
print(f"  Tokens/sec: {baseline_metrics['perf']['approx_tokens_per_sec']:.0f}")
print(f"  Peak memory: {baseline_metrics['perf']['max_mem_mb']:.0f}MB")

In [ ]:
# Create a "slow" config (simulating a regression)
slow_config = """
name: dpo_slow_profiled
description: Intentionally slow config to demonstrate regression detection

# Model configuration
model_name: EleutherAI/pythia-70m
use_peft: true
lora_r: 16
lora_alpha: 32
lora_dropout: 0.05

# Training configuration - SMALLER BATCH = SLOWER
training_type: dpo
max_steps: 80
batch_size: 1              # Reduced from 2!
gradient_accumulation_steps: 8  # Increased to compensate
learning_rate: 5.0e-5
max_length: 256
warmup_steps: 10

# DPO-specific
beta: 0.1
max_prompt_length: 64

# Dataset configuration
dataset_name: Anthropic/hh-rlhf
dataset_split: train
dataset_size: 512
seed: 42

# Output configuration
output_dir: ./profiler_output
metrics_warmup_steps: 10

# PROFILER to diagnose the slowdown
profiler:
  enabled: true
  start_step: 50
  num_steps: 20
  output_dir: ./profiler_traces_slow
  record_shapes: true
  profile_memory: true
  with_stack: false
"""

with open('configs/dpo_slow_profiled.yaml', 'w') as f:
    f.write(slow_config)

print("Created slow config with batch_size=1 (was 2)")
print("This will cause a throughput regression...")

In [ ]:
# Run the slow config
!python -m canary.cli run configs/dpo_slow_profiled.yaml -o ./profiler_output/slow_run

In [ ]:
# Compare the profiler outputs
slow_metrics_paths = list(Path('./profiler_output/slow_run').rglob('metrics.json'))
if not slow_metrics_paths:
    raise FileNotFoundError("No metrics.json found for slow run. Did the training complete?")

slow_metrics_path = slow_metrics_paths[0]
with open(slow_metrics_path) as f:
    slow_metrics = json.load(f)

print("\n" + "="*60)
print("COMPARISON: Baseline vs Slow Run")
print("="*60)

print(f"\n{'Metric':<25} {'Baseline':>15} {'Slow':>15} {'Change':>15}")
print("-"*70)

# Step time
base_step = baseline_metrics['perf']['step_time']['mean']
slow_step = slow_metrics['perf']['step_time']['mean']
change = (slow_step - base_step) / base_step * 100
print(f"{'Step time (s)':<25} {base_step:>15.4f} {slow_step:>15.4f} {change:>+14.1f}%")

# Tokens/sec
base_tps = baseline_metrics['perf']['approx_tokens_per_sec']
slow_tps = slow_metrics['perf']['approx_tokens_per_sec']
change = (slow_tps - base_tps) / base_tps * 100
print(f"{'Tokens/sec':<25} {base_tps:>15.0f} {slow_tps:>15.0f} {change:>+14.1f}%")

# Memory
base_mem = baseline_metrics['perf']['max_mem_mb']
slow_mem = slow_metrics['perf']['max_mem_mb']
change = slow_mem - base_mem
print(f"{'Memory (MB)':<25} {base_mem:>15.0f} {slow_mem:>15.0f} {change:>+14.0f}")

In [ ]:
# Compare profiler details
if 'profiler' in slow_metrics and slow_metrics['profiler']:
    slow_profiler = slow_metrics['profiler']
    base_profiler = baseline_metrics.get('profiler', {})
    
    print("\n" + "="*60)
    print("PROFILER COMPARISON")
    print("="*60)
    
    print(f"\n{'Metric':<25} {'Baseline':>15} {'Slow':>15}")
    print("-"*55)
    
    base_cuda = base_profiler.get('cuda_time_total_ms', 0)
    slow_cuda = slow_profiler.get('cuda_time_total_ms', 0)
    print(f"{'CUDA time (ms)':<25} {base_cuda:>15.2f} {slow_cuda:>15.2f}")
    
    base_cpu = base_profiler.get('cpu_time_total_ms', 0)
    slow_cpu = slow_profiler.get('cpu_time_total_ms', 0)
    print(f"{'CPU time (ms)':<25} {base_cpu:>15.2f} {slow_cpu:>15.2f}")
    
    print("\n" + "="*60)
    print("DIAGNOSIS")
    print("="*60)
    
    print("\nThe slowdown is caused by smaller batch size:")
    print("- More Python/framework overhead per sample")
    print("- Less efficient GPU utilization (smaller kernels)")
    print("- More dataloader calls per effective batch")
    print("\nFix: Increase batch_size, use gradient accumulation wisely")

## 9. Summary

### Key Takeaways:

1. **Enable profiling** with `profiler.enabled: true` in your config
2. **Skip warmup steps** - profile steady-state (start_step: 50+)
3. **Keep it short** - 20 steps is usually enough
4. **Check the ratio** - CPU time >> CUDA time = bottleneck
5. **Watch for memory copies** - they indicate sync issues
6. **Use TensorBoard** for detailed timeline analysis

### When to Profile:

- After a performance regression is detected
- When optimizing training code
- When debugging "my run got slower after N steps"
- Before and after major code changes

### Next Steps:

- See `03_stability_monitoring.ipynb` for detecting training instabilities
- See `04_root_cause_analysis.ipynb` for the full regression debugging workflow